In [1]:
import os
import torch

import scanpy as sc
import numpy as np
import pandas as pd
import gseapy as gp

from tqdm import tqdm
from sklearn.cluster import KMeans
from contrastive_vi.model.contrastive_vi import ContrastiveVIModel
from scripts import constants
from scvi._settings import settings

Global seed set to 0
During startup - Warning messages:
1: package ‘methods’ was built under R version 3.6.1 
2: package ‘datasets’ was built under R version 3.6.1 
3: package ‘utils’ was built under R version 3.6.1 
4: package ‘grDevices’ was built under R version 3.6.1 
5: package ‘graphics’ was built under R version 3.6.1 
6: package ‘stats’ was built under R version 3.6.1 


In [2]:
settings.seed = 0
device = "cuda:1"
dataset = "mcfarland_2020"

Global seed set to 0


In [3]:
pathway_enr_fdr = 0.05
expression_delta = 0.15

In [4]:
split_key = constants.DATASET_SPLIT_LOOKUP[dataset]["split_key"]
background_value = constants.DATASET_SPLIT_LOOKUP[dataset]["background_value"]
label_key = constants.DATASET_SPLIT_LOOKUP[dataset]["label_key"]
seeds = constants.DEFAULT_SEEDS
latent_size = 10

In [5]:
adata = sc.read_h5ad(
    os.path.join(
        constants.DEFAULT_DATA_PATH,
        f"{dataset}/preprocessed/adata_top_2000_genes_tc.h5ad",
    )
)
ContrastiveVIModel.setup_anndata(adata, layer="count")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


INFO     No batch_key inputted, assuming all cells are same batch                            
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["count"]                                               
INFO     Successfully registered anndata object containing 5928 cells, 2000 vars, 1 batches, 
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


In [6]:
target_indices = np.where(adata.obs[split_key] != background_value)[0]
background_indices = np.where(adata.obs[split_key] == background_value)[0]

In [7]:
genes = pd.read_table(
    os.path.join(
        constants.DEFAULT_DATA_PATH,
        dataset,
        "idasanutlin",
        "Idasanutlin_24hr_expt1",
        "genes.tsv",
    ),
    header=None,
)
genes = genes.rename(columns={0: "ensembl_id", 1: "gene_symbol"})
genes = genes[genes["ensembl_id"].isin(adata.var.index)]

In [8]:
model_list = []
for seed in tqdm(seeds):
    result_dir = os.path.join(
        constants.DEFAULT_RESULTS_PATH,
        f"{dataset}/contrastiveVI/latent_{latent_size}",
        f"{seed}",
    )
    model_list.append(
        torch.load(
            os.path.join(result_dir, "model.ckpt"),
            map_location=device,
        ),
    )

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.69s/it]


In [9]:
de_result_list = []
enr_result_list = []

for seed_index, seed in enumerate(seeds):
    model = model_list[seed_index]
    
    de_result = model.differential_expression(
        adata=adata,
        groupby=None,
        group1=None,
        group2=None,
        idx1=background_indices,
        idx2=target_indices,
        mode="change",
        delta=expression_delta,
        batch_size=128,
        all_stats=True,
        batch_correction=False,
        batchid1=None,
        batchid2=None,
        fdr_target=0.05,
        silent=False,
    )

    de_result.reset_index()
    de_result["ensembl_id"] = de_result.index
    de_result = de_result.merge(genes, on="ensembl_id")
    de_result["seed"] = seed
    de_result_list.append(de_result)

    top_genes = de_result[de_result["proba_de"] > 0.95]["gene_symbol"].tolist()
    enr = gp.enrichr(
        gene_list=top_genes,
        gene_sets="KEGG_2016",
        organism="human",
        cutoff=pathway_enr_fdr,
    )
    enr_result = enr.results
    enr_result = enr_result[enr_result["Adjusted P-value"] < pathway_enr_fdr]
    enr_result["seed"] = seed
    enr_result_list.append(enr_result)

DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.29s/it]


/tmp/ipykernel_21940/1947209614.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.21s/it]


/tmp/ipykernel_21940/1947209614.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.71s/it]


/tmp/ipykernel_21940/1947209614.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.78s/it]


/tmp/ipykernel_21940/1947209614.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.10s/it]


/tmp/ipykernel_21940/1947209614.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


In [10]:
enr_df = pd.concat(enr_result_list)
cols = ["Gene_set", "Term", "Adjusted P-value", "Overlap", "Genes"]
cols += ["seed"]
enr_df = enr_df[cols]

In [11]:
enr_df

,Gene_set,Term,Adjusted P-value,Overlap,Genes,seed
0,KEGG_2016,Rheumatoid arthritis Homo sapiens hsa05323,0.000482,18/90,CXCL6;IL11;HLA-DRB5;MMP1;TGFB3;CCL20;MMP3;CXCL...,123
1,KEGG_2016,Amoebiasis Homo sapiens hsa05146,0.001148,18/100,SERPINB3;SERPINB4;ARG2;SERPINB1;LAMB3;TGFB3;IL...,123
2,KEGG_2016,Cytokine-cytokine receptor interaction Homo sa...,0.002755,32/265,CXCL6;CSF3;IL26;IL24;TNFRSF11B;CXCL1;CXCL13;CX...,123
3,KEGG_2016,Protein digestion and absorption Homo sapiens ...,0.007718,15/90,CELA3A;PRSS1;COL15A1;COL27A1;COL1A1;COL3A1;SLC...,123
4,KEGG_2016,Transcriptional misregulation in cancer Homo s...,0.008656,23/180,NGFR;CDKN1A;SLC45A3;CDKN2C;HPGD;IL1R2;IGFBP3;Z...,123
...,...,...,...,...,...,...
12,KEGG_2016,Chemokine signaling pathway Homo sapiens hsa04062,0.026522,13/187,CCL20;CXCL1;CXCL13;CXCL14;GNG11;CXCL5;CXCL16;C...,999
13,KEGG_2016,Mineral absorption Homo sapiens hsa04978,0.031851,6/51,SLC34A2;MT1A;MT1M;MT1G;MT1H;MT1E,999
14,KEGG_2016,p53 signaling pathway Homo sapiens hsa04115,0.031851,7/69,CCNB1;CCND2;IGFBP3;SERPINE1;CDK1;SFN;GTSE1,999
15,KEGG_2016,Gap junction Homo sapiens hsa04540,0.031851,8/88,GUCY1A3;PDGFRB;GJA1;TUBA1A;PLCB4;ITPR1;CDK1;TU...,999


## Aggregate analysis

In [12]:
de_result = pd.concat(de_result_list)
de_result_mean = (
    de_result.groupby("gene_symbol", as_index=False)
    .mean()
    .sort_values(by="proba_de", ascending=False)
)

In [13]:
de_result_mean["proba_de"].describe()

count    2000.000000
mean        0.947154
std         0.014321
min         0.851120
25%         0.941840
50%         0.950160
75%         0.956400
max         0.971360
Name: proba_de, dtype: float64

In [14]:
top_genes = de_result_mean[de_result_mean["proba_de"] > 0.95]["gene_symbol"].tolist()

enr = gp.enrichr(
    gene_list=top_genes,
    gene_sets="KEGG_2016",
    organism="human",
    cutoff=0.05,
)
enr_results = enr.results
enr_results = enr_results[enr_results["Adjusted P-value"] < 0.05]

In [15]:
enr_results

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,KEGG_2016,Rheumatoid arthritis Homo sapiens hsa05323,18/90,5.691453e-07,0.000129,0,0,4.702736,67.621257,CXCL6;IL11;HLA-DRB5;MMP1;TGFB3;CCL20;MMP3;CXCL...
1,KEGG_2016,Amoebiasis Homo sapiens hsa05146,18/100,2.839565e-06,0.000321,0,0,4.127048,52.710074,SERPINB3;SERPINB4;ARG2;SERPINB1;LAMB3;TGFB3;IL...
2,KEGG_2016,Cytokine-cytokine receptor interaction Homo sa...,30/265,3.988912e-05,0.002196,0,0,2.409462,24.406421,CXCL6;IL26;IL24;TNFRSF11B;CXCL1;CXCL13;CXCL3;C...
3,KEGG_2016,Protein digestion and absorption Homo sapiens ...,15/90,4.827853e-05,0.002196,0,0,3.750397,37.273408,COL17A1;PRSS1;COL15A1;COL11A1;COL1A1;COL3A1;SL...
4,KEGG_2016,Transcriptional misregulation in cancer Homo s...,23/180,4.858856e-05,0.002196,0,0,2.757070,27.383558,NGFR;CDKN1A;CDKN2C;JUP;HPGD;IL1R2;IGFBP3;ZBTB1...
5,KEGG_2016,ECM-receptor interaction Homo sapiens hsa04512,14/82,6.512975e-05,0.002453,0,0,3.858305,37.190697,LAMB3;ITGA2;LAMA4;TNC;FN1;LAMC2;THBS2;THBS1;CO...
6,KEGG_2016,TNF signaling pathway Homo sapiens hsa04668,15/110,4.784629e-04,0.015448,0,0,2.957707,22.611467,EDN1;CCL20;MMP3;CXCL1;CXCL3;PTGS2;MMP9;CXCL5;I...
7,KEGG_2016,Graft-versus-host disease Homo sapiens hsa05332,8/41,9.644948e-04,0.027247,0,0,4.524616,31.418506,HLA-DRB5;IL6;IL1B;PRF1;GZMB;KLRC1;HLA-G;HLA-DRB1
8,KEGG_2016,Pertussis Homo sapiens hsa05133,11/75,1.442221e-03,0.036216,0,0,3.212126,21.012354,PYCARD;CXCL6;IL6;CALML5;SFTPA2;C1S;IL1B;CALML3...
9,KEGG_2016,Salivary secretion Homo sapiens hsa04970,12/89,1.881268e-03,0.042517,0,0,2.913407,18.283988,GUCY1A3;BST1;CST1;PLCB4;CALML5;DMBT1;KCNMA1;IT...


In [16]:
len(top_genes)

1023